In [1]:
!pip install datasets  

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from datasets import load_dataset


**Load CIFAR from hugging face**

In [2]:
# Load the CIFAR-10 dataset from Hugging Face
ds = load_dataset("cifar10")

# Separate train and test splits
train_ds = ds["train"]
test_ds = ds["test"]

def preprocess(dataset):
    images = []
    labels = []
    for item in dataset:
        # item["img"] is a PIL Image; convert to NumPy array and normalize to [0,1]
        img = np.array(item["img"]).astype("float32") / 255.0
        images.append(img)
        labels.append(item["label"])
    images = np.stack(images)
    labels = np.array(labels)
    return images, labels

# Preprocess train and test datasets
x_train, y_train = preprocess(train_ds)
x_test, y_test = preprocess(test_ds)

print("x_train shape:", x_train.shape)  # Expected: (50000, 32, 32, 3)
print("x_test shape:", x_test.shape)    # Expected: (10000, 32, 32, 3)


README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)


**One Hot encode labels**

In [3]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)

print("y_train shape:", y_train.shape)  # Expected: (50000, 10)
print("y_test shape:", y_test.shape)    # Expected: (10000, 10)


y_train shape: (50000, 10)
y_test shape: (10000, 10)


Model 

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
import matplotlib.pyplot as plt

# Define the CNN using the Functional API
inp = Input(shape=(32, 32, 3))  # explicit input
x = layers.Conv2D(32, (3, 3), activation='relu')(inp)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

# Create the functional model
model_func = Model(inputs=inp, outputs=outputs)
model_func.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          65,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 122,570 (478.79 KB)

 Trainable params: 122,570 (478.79 KB)

 Non-trainable params: 0 (0.00 B)

**Without data augmentation**

In [5]:
model_func.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_no_aug = model_func.fit(x_train, y_train,
                           epochs=20,
                           batch_size=64,
                           validation_data=(x_test, y_test))


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 47ms/step - accuracy: 0.3151 - loss: 1.8351 - val_accuracy: 0.5142 - val_loss: 1.3551
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 35s 44ms/step - accuracy: 0.5271 - loss: 1.3215 - val_accuracy: 0.5541 - val_loss: 1.2736
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 35s 44ms/step - accuracy: 0.5912 - loss: 1.1648 - val_accuracy: 0.6244 - val_loss: 1.0663
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 44ms/step - accuracy: 0.6339 - loss: 1.0446 - val_accuracy: 0.6526 - val_loss: 0.9984
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.6636 - loss: 0.9647 - val_accuracy: 0.6619 - val_loss: 0.9527
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 44ms/step - accuracy: 0.6831 - loss: 0.9083 - val_accuracy: 0.6672 - val_loss: 0.9459
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 42ms/step - accuracy: 0.7014 - loss: 0.8570 - val_accuracy: 0.6731 - val_loss: 0.9375
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 44ms/step - accuracy: 0.7131 - loss: 0.8223 - 

**With Data Augmentation**

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Fit the generator on the training data (optional for CIFAR-10)
datagen.fit(x_train)

# Build a new model instance (same architecture)
model_aug = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=x_train.shape[1:]),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model_aug.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history_aug = model_aug.fit(datagen.flow(x_train, y_train, batch_size=64),
                            epochs=20,
                            validation_data=(x_test, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 63s 78ms/step - accuracy: 0.3046 - loss: 1.8575 - val_accuracy: 0.4786 - val_loss: 1.4210
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 61s 78ms/step - accuracy: 0.4980 - loss: 1.3889 - val_accuracy: 0.5728 - val_loss: 1.2030
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 59s 75ms/step - accuracy: 0.5563 - loss: 1.2454 - val_accuracy: 0.5958 - val_loss: 1.1486
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 58s 74ms/step - accuracy: 0.5844 - loss: 1.1631 - val_accuracy: 0.6358 - val_loss: 1.0417
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 58s 74ms/step - accuracy: 0.6125 - loss: 1.1020 - val_accuracy: 0.6318 - val_loss: 1.0570
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 74ms/step - accuracy: 0.6251 - loss: 1.0556 - val_accuracy: 0.6726 - val_loss: 0.9340
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 57s 72ms/step - accuracy: 0.6383 - loss: 1.0199 - val_accuracy: 0.6503 - val_loss: 1.0464
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 56s 71ms/step - accuracy: 0.6541 - loss: 0.9810 - val_accurac

**Comaprison**

In [7]:
# Evaluate the model without augmentation
test_loss_no_aug, test_acc_no_aug = model.evaluate(x_test, y_test)
print("Test Accuracy without augmentation:", test_acc_no_aug)

# Evaluate the model with augmentation
test_loss_aug, test_acc_aug = model_aug.evaluate(x_test, y_test)
print("Test Accuracy with augmentation:", test_acc_aug)

# Plot loss and accuracy curves for both models
plt.figure(figsize=(12,5))

# Loss curves
plt.subplot(1,2,1)
plt.plot(history_no_aug.history['loss'], label='Train Loss (No Aug)')
plt.plot(history_no_aug.history['val_loss'], label='Val Loss (No Aug)')
plt.plot(history_aug.history['loss'], label='Train Loss (Aug)')
plt.plot(history_aug.history['val_loss'], label='Val Loss (Aug)')
plt.title("Loss Curves")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

# Accuracy curves
plt.subplot(1,2,2)
plt.plot(history_no_aug.history['accuracy'], label='Train Acc (No Aug)')
plt.plot(history_no_aug.history['val_accuracy'], label='Val Acc (No Aug)')
plt.plot(history_aug.history['accuracy'], label='Train Acc (Aug)')
plt.plot(history_aug.history['val_accuracy'], label='Val Acc (Aug)')
plt.title("Accuracy Curves")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.tight_layout()
plt.show()

NameError: name 'model' is not defined

**Feature Map Visualization**

In [ ]:
# Get outputs of all Conv2D layers
conv_layer_outputs = [layer.output for layer in model_func.layers if isinstance(layer, tf.keras.layers.Conv2D)]

# Create a feature map model that takes the same input and outputs these intermediate activations
feature_map_model = Model(inputs=model_func.input, outputs=conv_layer_outputs)

# For example, take an example image from the test set (make sure x_test is defined)
example_image = x_test[0:1]  # shape: (1, 32, 32, 3)
feature_maps = feature_map_model.predict(example_image)

# Print the shape of the first convolutional layer’s output
print("Feature maps from first conv layer:", feature_maps[0].shape)

# Visualize the feature maps of the first Conv2D layer
num_filters = feature_maps[0].shape[-1]
plt.figure(figsize=(20, 8))
for i in range(num_filters):
    plt.subplot(4, num_filters//4, i+1)
    plt.imshow(feature_maps[0][0, :, :, i], cmap='viridis')
    plt.axis('off')
plt.suptitle("Feature Maps from First Convolutional Layer")
plt.show()


In [ ]:
# Assuming you have already created feature_map_model and obtained feature_maps from an example image:
# feature_maps = feature_map_model.predict(example_image)

# Get feature maps from the deeper (third) convolutional layer
deeper_conv_maps = feature_maps[2]  # Index 2 corresponds to the third conv layer
print("Feature maps from third conv layer:", deeper_conv_maps.shape)

# Visualize the feature maps from the third convolutional layer
num_filters = deeper_conv_maps.shape[-1]
plt.figure(figsize=(20, 8))
for i in range(num_filters):
    plt.subplot(4, num_filters // 4, i+1)
    plt.imshow(deeper_conv_maps[0, :, :, i], cmap='viridis')
    plt.axis('off')
plt.suptitle("Feature Maps from Third (Deeper) Convolutional Layer")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
# Get predicted class labels for the test set
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=["Airplane", "Automobile", "Bird", "Cat", "Deer",
                         "Dog", "Frog", "Horse", "Ship", "Truck"],
            yticklabels=["Airplane", "Automobile", "Bird", "Cat", "Deer",
                         "Dog", "Frog", "Horse", "Ship", "Truck"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix Heatmap")
plt.show()


In [ ]:
# Save the model to a file named 'my_cnn_model.h5'
model.save('my_cnn_model.h5')
print("Model saved successfully!")


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
saved_model = load_model('my_cnn_model.h5')
print("Model loaded successfully!")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define class names for CIFAR-10
class_names = ["Airplane", "Automobile", "Bird", "Cat", "Deer",
               "Dog", "Frog", "Horse", "Ship", "Truck"]

# Select a few test images (for example, the first 5)
num_examples = 5
sample_images = x_test[:num_examples]
sample_labels = y_test[:num_examples]

# Run predictions on the selected test images
predictions = saved_model.predict(sample_images)

# Convert predictions and true labels from one-hot to class indices
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(sample_labels, axis=1)

# Plot the test images with predicted and true labels
plt.figure(figsize=(15, 5))
for i in range(num_examples):
    plt.subplot(1, num_examples, i+1)
    plt.imshow(sample_images[i])
    plt.title(f"Predicted: {class_names[predicted_classes[i]]}\nTrue: {class_names[true_classes[i]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Assuming you have already loaded your saved model into 'saved_model'
test_loss, test_accuracy = saved_model.evaluate(x_test, y_test, verbose=2)
print("Test accuracy: {:.2f}%".format(test_accuracy * 100))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert one-hot test labels to integer labels
y_true = np.argmax(y_test, axis=1)

# Generate predictions for each model
y_pred_no_aug = np.argmax(model_func.predict(x_test), axis=1)
y_pred_aug = np.argmax(model_aug.predict(x_test), axis=1)

# Compute metrics for the model without augmentation
acc_no_aug = accuracy_score(y_true, y_pred_no_aug)
prec_no_aug = precision_score(y_true, y_pred_no_aug, average='macro')
rec_no_aug = recall_score(y_true, y_pred_no_aug, average='macro')
f1_no_aug = f1_score(y_true, y_pred_no_aug, average='macro')

# Compute metrics for the model with augmentation
acc_aug = accuracy_score(y_true, y_pred_aug)
prec_aug = precision_score(y_true, y_pred_aug, average='macro')
rec_aug = recall_score(y_true, y_pred_aug, average='macro')
f1_aug = f1_score(y_true, y_pred_aug, average='macro')


In [ ]:
import pandas as pd

data = {
    "Accuracy": [acc_no_aug, acc_aug],
    "Precision": [prec_no_aug, prec_aug],
    "Recall": [rec_no_aug, rec_aug],
    "F1-Score": [f1_no_aug, f1_aug]
}
df_results = pd.DataFrame(data, index=["Without Augmentation", "With Augmentation"])
print(df_results)


In [ ]:
import matplotlib.pyplot as plt

# Prepare metric names and values for the bar chart
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
no_aug_vals = [acc_no_aug, prec_no_aug, rec_no_aug, f1_no_aug]
aug_vals = [acc_aug, prec_aug, rec_aug, f1_aug]

# Create a figure with multiple subplots
plt.figure(figsize=(18, 12))

# Subplot 3: Bar chart for overall performance metrics
plt.subplot(2,2,3)
x = np.arange(len(metrics))  # positions for groups
width = 0.35  # width of the bars
plt.bar(x - width/2, no_aug_vals, width, label='No Augmentation')
plt.bar(x + width/2, aug_vals, width, label='With Augmentation')
plt.xticks(x, metrics)
plt.ylabel('Score')
plt.ylim([0, 1])
plt.title("Performance Metrics Comparison")
plt.legend()

plt.tight_layout()
plt.show()


# Ablation Study

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def build_cnn_model(num_filters=32, num_conv_layers=3):
    """
    Builds a simple CNN model with variable number of filters
    and convolutional layers.
    
    Parameters:
      num_filters (int): Number of filters in the first Conv layer
      num_conv_layers (int): How many Conv + Pool layers to stack
      
    Returns:
      A compiled Keras model (untrained).
    """
    model = Sequential()
    
    # First conv layer with input_shape
    model.add(Conv2D(num_filters, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    
    # Add additional conv/pool layers
    current_filters = num_filters
    for _ in range(num_conv_layers - 1):
        current_filters *= 2  # Optionally double the filters each time
        model.add(Conv2D(current_filters, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    return model


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

learning_rates = [0.001, 0.01, 0.1]
lr_results = {}

for lr in learning_rates:
    # Build a new model
    model_lr = build_cnn_model(num_filters=32, num_conv_layers=3)
    
    # Compile with a custom learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_lr.compile(optimizer=optimizer,
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
    
    # Train briefly (5 epochs for demonstration)
    history_lr = model_lr.fit(x_train, y_train,
                              epochs=5,
                              batch_size=64,
                              validation_data=(x_test, y_test),
                              verbose=0)  # silent training
    
    # Evaluate on test data
    test_loss, test_acc = model_lr.evaluate(x_test, y_test, verbose=0)
    
    lr_results[lr] = test_acc
    print(f"Learning Rate {lr} => Test Accuracy: {test_acc:.4f}")

# Convert dictionary to lists (sorted by learning rate for clarity)
lr_vals = sorted(lr_results.keys())
acc_vals = [lr_results[lr] for lr in lr_vals]

plt.figure(figsize=(8,6))
plt.plot(lr_vals, acc_vals, marker='o', linestyle='-')
plt.xscale('log')
plt.xlabel('Learning Rate (log scale)')
plt.ylabel('Test Accuracy')
plt.title('Learning Rate vs. Test Accuracy')
plt.grid(True)
plt.show()

In [ ]:
batch_sizes = [16, 32, 64]
batch_results = {}

for bs in batch_sizes:
    model_bs = build_cnn_model(num_filters=32, num_conv_layers=3)
    model_bs.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
    
    history_bs = model_bs.fit(x_train, y_train,
                              epochs=5,
                              batch_size=bs,
                              validation_data=(x_test, y_test),
                              verbose=0)
    
    test_loss, test_acc = model_bs.evaluate(x_test, y_test, verbose=0)
    
    batch_results[bs] = test_acc
    print(f"Batch Size {bs} => Test Accuracy: {test_acc:.4f}")

import matplotlib.pyplot as plt

# Convert dictionary to lists (sorted by batch size)
batch_vals = sorted(batch_results.keys())
acc_vals = [batch_results[bs] for bs in batch_vals]

plt.figure(figsize=(8,6))
plt.plot(batch_vals, acc_vals, marker='o', linestyle='-')
plt.xlabel('Batch Size')
plt.ylabel('Test Accuracy')
plt.title('Batch Size vs. Test Accuracy')
plt.grid(True)
plt.show()


In [ ]:
filter_options = [16, 32, 64]
filter_results = {}

for f in filter_options:
    model_f = build_cnn_model(num_filters=f, num_conv_layers=3)
    model_f.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
    
    history_f = model_f.fit(x_train, y_train,
                            epochs=5,
                            batch_size=64,
                            validation_data=(x_test, y_test),
                            verbose=0)
    
    test_loss, test_acc = model_f.evaluate(x_test, y_test, verbose=0)
    
    filter_results[f] = test_acc
    print(f"Num Filters (first layer) {f} => Test Accuracy: {test_acc:.4f}")

import matplotlib.pyplot as plt

# Convert dictionary to lists (sorted by filter options)
filter_vals = sorted(filter_results.keys())
acc_vals = [filter_results[f] for f in filter_vals]

plt.figure(figsize=(8,6))
plt.plot(filter_vals, acc_vals, marker='o', linestyle='-')
plt.xlabel('Number of Filters in First Layer')
plt.ylabel('Test Accuracy')
plt.title('Number of Filters vs. Test Accuracy')
plt.grid(True)
plt.show()


In [ ]:
layer_options = [3]
layer_results = {}

for nlayers in layer_options:
    model_layers = build_cnn_model(num_filters=32, num_conv_layers=nlayers)
    model_layers.compile(optimizer='adam',
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])
    
    history_layers = model_layers.fit(x_train, y_train,
                                      epochs=5,
                                      batch_size=64,
                                      validation_data=(x_test, y_test),
                                      verbose=0)
    
    test_loss, test_acc = model_layers.evaluate(x_test, y_test, verbose=0)
    
    layer_results[nlayers] = test_acc
    print(f"Num Convolutional Layers {nlayers} => Test Accuracy: {test_acc:.4f}")
